<a href="https://colab.research.google.com/github/A-K-0802/GEN-AI-2025/blob/main/Currency_converter_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [121]:
!pip install -q -U langchain-google-genai langchain-core requests

In [122]:
from langchain_google_genai import ChatGoogleGenerativeAI
import  os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"]=userdata.get("GOOGLE_API_KEY")

llm=ChatGoogleGenerativeAI(
    model="gemini-2.5-flash"
)

In [123]:
from langchain_core.tools import tool
from langchain_core.tools import InjectedToolArg
from typing import Annotated
import requests
@tool
def get_conversion_rates(base_currency: str, target_currency:str) -> float:
    """Get conversion rates for a given base currency to given target currency."""
    url=f"https://v6.exchangerate-api.com/v6/85e9dcbfe0931ec9d8a57b16/pair/{base_currency}/{target_currency}"
    response=requests.get(url)
    return response.json()



@tool
def convert(base_value:int,conversion_rate:Annotated[float,InjectedToolArg])->float:
  """Given a currency conversion rate this function calculates the target currency value from a given base currency value"""
  return base_value*conversion_rate



In [124]:
output=get_conversion_rates.invoke({'base_currency':'USD','target_currency':'INR'})
output

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1750291201,
 'time_last_update_utc': 'Thu, 19 Jun 2025 00:00:01 +0000',
 'time_next_update_unix': 1750377601,
 'time_next_update_utc': 'Fri, 20 Jun 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 86.4882}

In [125]:
convert.invoke({'base_value':10,'conversion_rate':86.18})

861.8000000000001

In [126]:
llm_with_tools=llm.bind_tools([get_conversion_rates,convert])


In [127]:
from langchain_core.messages import HumanMessage
messages=[HumanMessage("What is conversion factor between USD and INR and based on that CONVERT 10 USD to INR")]

In [128]:
response=llm_with_tools.invoke(messages)
messages.append(response)
response.tool_calls

[{'name': 'get_conversion_rates',
  'args': {'target_currency': 'INR', 'base_currency': 'USD'},
  'id': '27ce616b-724e-4ade-99a0-d37fe119f340',
  'type': 'tool_call'}]

In [129]:
import json
for toolcall in response.tool_calls:
  if toolcall['name']=="get_conversion_rates":
    tool_message1=get_conversion_rates.invoke(toolcall)
    conv_rate=json.loads(tool_message1.content)['conversion_rate']
    messages.append(tool_message1)
  if toolcall['name']=="convert":
    toolcall['args']['conversion_rate']=conv_rate
    tool_message2=convert.invoke(toolcall)
    messages.append(tool_message2)


In [130]:
messages

[HumanMessage(content='What is conversion factor between USD and INR and based on that CONVERT 10 USD to INR', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_conversion_rates', 'arguments': '{"target_currency": "INR", "base_currency": "USD"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'models/gemini-2.5-flash-preview-05-20', 'safety_ratings': []}, id='run--e4510581-214f-4975-b9fd-805350446186-0', tool_calls=[{'name': 'get_conversion_rates', 'args': {'target_currency': 'INR', 'base_currency': 'USD'}, 'id': '27ce616b-724e-4ade-99a0-d37fe119f340', 'type': 'tool_call'}], usage_metadata={'input_tokens': 139, 'output_tokens': 26, 'total_tokens': 297, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 132}}),
 ToolMessage(content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms

In [131]:
llm_with_tools.invoke(messages).content

'The conversion factor between USD and INR is 86.4882.\n\nBased on this, 10 USD is equivalent to 864.882 INR (10 * 86.4882).'